In [1]:
import os
import itertools
import json
import numpy as np


In [2]:
# ipyparallel
import ipyparallel

# get the clients
clients = ipyparallel.Client()
print(clients.ids)

# create a direct view into the engines
dview = clients.direct_view()

with dview.sync_imports():
    from code.trainer import compute_recording_distributions
    from code.trainer import train_single
    from code.trainer import train_multi 
 

[0, 1, 2, 3, 4, 5]
importing compute_recording_distributions from code.trainer on engine(s)
importing train_single from code.trainer on engine(s)
importing train_multi from code.trainer on engine(s)


In [3]:
# paths
dataset_path = os.path.abspath(os.path.join(
        '..', '..', 'test_datasets', 'ottoman_turkish_makam_recognition_dataset'))

# parameters
distribution_types = ["pd", "pcd"]
step_sizes = [7.5, 15.0, 25.0, 50.0, 100.0]
kernel_widths = [0, 7.5, 15.0, 25.0, 50.0, 100.0]
model_types = ['single', 'multi']

# load annotation
annotations = json.load(open(os.path.join(dataset_path, 'annotations.json')))

# ipyparallel executes the code in where the engines have started
# add the absolute repository directory to annotation
for a in annotations:
    a['dataset_path'] = dataset_path

# load folds
folds = json.load(open(os.path.join('data', 'folds.json')))

# add index to folds for saving
folds = list(enumerate(folds))


## FEATURE EXTRACTION

In [4]:
# compute the distribution per recording for all combinations
fcombs = list(
    itertools.product(step_sizes, kernel_widths, distribution_types, annotations))
# ignore combinations in which kernel_width is three times less than the step_size
fcombs = [c for c in fcombs if c[1] == 0 or 3 * c[1] >= c[0]]
fcombs = np.array(fcombs).transpose().tolist()

feature_result = dview.map_sync(compute_recording_distributions, *fcombs)


## TRAINING

In [5]:
# get all parameter combinations
tcombs = list(
    itertools.product(step_sizes, kernel_widths, distribution_types, folds))
# ignore combinations in which kernel_width is three times less than the step_size
tcombs = [c for c in tcombs if c[1] == 0 or 3 * c[1] >= c[0]]
tcombs = np.array(tcombs).transpose().tolist()

single_result = dview.map_sync(train_single, *tcombs)
multi_result = dview.map_sync(train_multi, *tcombs)